In [40]:
# SummaryLangGraph_Sync.ipynb

# Cell 1: Imports
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from typing import List, Dict, TypedDict

# ---- Load PDF ----
pdf_path = "Sample_Loan.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Define the shared state
class State(TypedDict):
    docs: List[Document]
    accumulated_summaries: List[str]
    current_summary: str
    final_summary: str

# Initialize the model
llm = Ollama(model="llama3.2")  # or "mistral" or any model you prefer

map_prompt = PromptTemplate.from_template(
    "Write an excellent summary of the following, covering every critical point:\n\n{context}"
)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_prompt = PromptTemplate.from_template(
    "The following is a set of summaries:\n{summaries}\nTake these and distill it into a final, consolidated summary of the main themes."
)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Define map node
def map_node(state: State) -> State:
    print("🧩 Map step")
    doc = state["docs"].pop(0)
    # print(f"   - Processing doc: {doc.page_content}")
    result = map_chain.invoke({"context": doc.page_content})
    # print(f"   ✅ Got summary: {result['text']}")
    state["accumulated_summaries"].append(result["text"])
    return state

# Define reduce node
def reduce_node(state: State) -> State:
    print("\n🧪 Reduce step")
    combined = "\n".join(state["accumulated_summaries"])
    result = reduce_chain.invoke({"summaries": combined})
    # print(f"   ✅ Final summary: {result['text']}")
    state["final_summary"] = result["text"]
    return state

# Define condition for looping
def should_continue(state: State) -> str:
    return "map" if len(state["docs"]) > 0 else "reduce"

# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("map", map_node)
graph_builder.add_node("reduce", reduce_node)
graph_builder.set_conditional_entry_point(should_continue)
graph_builder.add_edge("map", "reduce")
graph_builder.add_edge("reduce", END)
graph = graph_builder.compile()

# Run it synchronously
initial_state: State = {
    "docs": docs.copy(),
    "accumulated_summaries": [],
    "current_summary": "",
    "final_summary": "",
}

print("🚀 Starting summarization...\n")
final_state = graph.invoke(initial_state)
print("\n✅ Final Summary Output:\n")
print(final_state["final_summary"])


🚀 Starting summarization...

🧩 Map step

🧪 Reduce step

✅ Final Summary Output:

Here's a consolidated summary of the key points from the loan agreement and promissory note:

**Loan Agreement and Promissory Note Summary**

A loan agreement and promissory note were executed on July 1, 2010, between Wharton Capital, LLC (the "Lender") and Sangrine Corp (the "Borrower"). The key points are as follows:

*   Loan amount: $27,500.00
*   Interest rate: 7% per annum
*   Repayment terms:
    *   Three separate payments will be made according to a specific schedule
    *   No details provided for the first and second payment amounts
    *   No details provided for the third payment amount

Please note that this summary only covers the critical points mentioned in the original text and may not include all aspects of the loan agreement and promissory note.
